In [32]:
import keras
from keras.layers import Dense, Conv2D, Flatten
from keras.models import Sequential, load_model
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.utils import to_categorical, plot_model
from keras.preprocessing.image import ImageDataGenerator
import pydot

In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv('./df_final.csv')

mask = np.random.rand(len(data)) < 0.8
train = data[mask].reset_index(drop=True)
test = data[~mask].reset_index(drop=True)

In [14]:
height = 224
width = 224

In [12]:
gen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                             rotation_range=30, 
                             vertical_flip=True, 
                             validation_split=0.2)

In [15]:
testgen = gen.flow_from_dataframe(test,
                                  directory='./images/comp/',
                                  x_col='fileid',
                                  y_col='Type1',
                                  class_mode='categorical',
                                  target_size=(height, width))

Found 5073 images belonging to 18 classes.


In [2]:
model = load_model('./checkpoints/ResNet50_model_weights.h5')

In [3]:
type(model)

keras.engine.training.Model

In [19]:
predictions = model.predict_generator(testgen, steps=1250, verbose=1)

1250/1250 [==============================] - 5093s 4s/step


In [31]:
predictions

(39895, 18)

In [ ]:
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")

In [33]:
print('Predicted:', decode_predictions(predictions, top=3)[0])
np.argmax(pred[0])

ValueError: `decode_predictions` expects a batch of predictions (i.e. a 2D array of shape (samples, 1000)). Found array with shape: (39895, 18)

In [30]:
plot_model(model, to_file='model.png')

ImportError: Failed to import `pydot`. Please install `pydot`. For example with `pip install pydot`.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, Activation
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, History
from keras.optimizers import SGD, Adam

data = pd.read_csv('./df_final.csv')

height = 224
width = 224

mask = np.random.rand(len(data)) < 0.8
train = data[mask].reset_index(drop=True)
test = data[~mask].reset_index(drop=True)

pretrainmod = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(height, width, 3))

gen = ImageDataGenerator(preprocessing_function=preprocess_input, rotation_range=30, vertical_flip=True, validation_split=0.2)

train_gen = gen.flow_from_dataframe(train,
                                    directory='./images/comp/',
                                    x_col='fileid',
                                    y_col='Type1',
                                    class_mode='categorical',
                                    target_size=(height, width),
                                    subset='training')


def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        # New FC layer, random init
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)

    # New softmax layer
    predictions = Dense(num_classes, activation='softmax')(x)

    finetune_model = Model(inputs=base_model.input, outputs=predictions)

    return finetune_model


class_list = list(data.Type1.unique())
FC_LAYERS = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(pretrainmod,
                                      dropout=dropout,
                                      fc_layers=FC_LAYERS,
                                      num_classes=len(class_list))


NUM_EPOCHS = 10
BATCH_SIZE = 16
num_train_images = 15000

adam = Adam(lr=0.00001)
finetune_model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

filepath = "./checkpoints/" + "ResNet50" + "_model_weights.h5"
histpath = "./checkpoints/" + "ResNet50" + "_model_history.h5"
checkpoint = ModelCheckpoint(filepath, monitor=["acc"], verbose=1, mode='max')
history = History()
callbacks_list = [checkpoint, history]

history = finetune_model.fit_generator(train_gen, epochs=NUM_EPOCHS, workers=8,
                                       steps_per_epoch=num_train_images // BATCH_SIZE,
                                       shuffle=True, callbacks=callbacks_list)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 16130 images belonging to 18 classes.
Epoch 1/10
937/937 [==============================] - 4423s 5s/step - loss: 2.8016 - acc: 0.1330

Epoch 00001: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 2/10
937/937 [==============================] - 4415s 5s/step - loss: 2.6426 - acc: 0.1831

Epoch 00002: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 3/10
937/937 [==============================] - 4351s 5s/step - loss: 2.5358 - acc: 0.2126

Epoch 00003: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 4/10
937/937 [==============================] - 4684s 5s/step - loss: 2.4614 - acc: 0.2441

Epoch 00004: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 5/10
937/937 [==============================] - 4294s 5s/step - loss: 2.3876 - acc: 0.2624

Epoch 00005: saving model to ./checkpoints/ResNet50_model_weights.h5
Epoch 6/10
937/937 [==============================] - 4229s 5s/step - loss: 2.3265 - acc: 0.2855

Epoch 00006: saving mod